In [ ]:
#compute the strneght of schedule - to do that we need two things - 
#the record (wins/total) for each team, and the list of teams a team has played up till then

#each week 
#for each game for each team (H/A)
#TR = wins/total
#OR = avg the TR for every team in "played-list"
#OOR= for ever team in "played-list" average the average of their OR

#BCS SOS = (2OR + OOR )/3
#RPI = .25 * TR + .5 * OR + .25 * OOR

#2nd pass - update wins, total, opponets list

import os
import pandas as pd
import numpy as np
import csv



class TeamRecord:
    def __init__(self, wins, total_games, opponents):
        self.wins = wins
        self.total_games = total_games
        self.opponents = opponents
    
    def __init__(self):
        self.wins = 0
        self.total_games = 0
        self.opponents = []
        
    def __str__(self):
        return str(self.wins) + ":" + str(self.total_games) + ":" + str(self.opponents)

def getTeam(recordList, teamid):
    
    if not teamid in recordList:
        team = TeamRecord()
        recordList[teamid] = team
    else:
        team = recordList[teamid]
    return team

def getRecord(team):
    return 0 if team.total_games == 0 else team.wins / team.total_games

def getRecordByID(recordList, teamID):
    team = getTeam(recordList, teamID)
    return getRecord(team)

def getORecord(recordList, team):
    total_record = 0
    for opp in team.opponents:
        total_record = getRecordByID(recordList, opp)
    return 0 if len(team.opponents) == 0 else total_record / len(team.opponents)

def getOORecord(recordList, team):
    total_record = 0
    for opp in team.opponents:
        total_record = getORecord(recordList, getTeam(recordList, opp))
    return 0 if len(team.opponents) == 0 else total_record / len(team.opponents)

    
    
def updateRecords(recordList, games):
    
    for game in games.itertuples():
        home = getTeam(recordList, game.HomeID)
        away = getTeam(recordList, game.VisID)

        home.total_games += 1
        away.total_games += 1
        
        if (game.HomeFinal > game.VisFinal): home.wins +=1 
        else: away.wins += 1
        
        home.opponents.append(game.VisID)
        away.opponents.append(game.HomeID)
        

def computeTeamSOS(recordList, team):
    teamRecord = getRecord(team)
    teamOppRecord = getORecord(recordList, team)
    teamOppOppRecord = getOORecord(recordList, team)
        
    teamBCSSOS = (2 * teamOppRecord + teamOppOppRecord) / 3
    teamRPI = .25 * teamRecord + .5 * teamOppRecord + .25 * teamOppOppRecord
    
    return teamBCSSOS, teamRPI
        
def computeWeekSOS(recordList, games):
    
    results = []
    for game in games.itertuples():
        home = getTeam(recordList, game.HomeID)
        away = getTeam(recordList, game.VisID)
            
        homeBCSSOS, homeRPI = computeTeamSOS(recordList, home)
        awayBCSSOS, awayRPI = computeTeamSOS(recordList, home)
            
        res = {}
        res['HomeID'] = game.HomeID
        res['VisID'] = game.VisID
        res['Season'] = game.Season
        res['Week'] = game.Week
        res['HomeBCSSOS'] = homeBCSSOS
        res['HomeRPI'] = homeRPI
        res['AwayBCSSOS'] = awayBCSSOS
        res['AwayRPI'] = awayRPI
        
        results.append(res)
        
    return results
        


root_dir = os.getcwd()
allGames = pd.read_csv(os.path.join(root_dir, "..", "data", "snoozle", "snoozle-combined.csv" ))

all_results = []

for year in (allGames.Season.unique()):
    
    thisSeason = allGames[allGames['Season'] == year]
    masterRecordList = {}
    
    for weeknum in thisSeason.Week.unique(): #change this to actual week values

        week = thisSeason[thisSeason['Week'] == weeknum]

        all_results += computeWeekSOS(masterRecordList, week) 
        updateRecords(masterRecordList, week)

    print("finished", year)

keys = all_results[0].keys()
with open(os.path.join(root_dir, "..", "data", "BCS", "BCS-SOS.csv"), 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_results)

print("Finished Writing Output")